# Lecture 13 Demos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy as sp

Let's define some arbitrary DT signal:

In [ ]:
def random_signal_generator(N):
    return (np.random.uniform(size=N) * np.arange(N) / N)

In [ ]:
plt.stem(random_signal_generator(100))

## DFT with matrix multiplication

In [ ]:
def construct_dft_matrix(N, method="fastest"):
    ζ = np.exp(-1j * 2 * np.pi / N)
    mat = np.ones((N, N), dtype=np.complex128)
    
    # Slowest method uses a nested for loop
    if method == "slow":
        for row_idx in range(N):
            for col_idx in range(N):
                mat[row_idx, col_idx] = ζ ** (row_idx * col_idx)

    # Slightly faster method leveraging the relationships between
    # rows of the DFT matrix.
    if method == "fast":
        mat[1] = np.array([ζ ** n for n in range(N)])
        for row in range(2, N):
            mat[row] = mat[1] ** row
            
    # Blazing fast: use the FFT! 
    if method == "fastest":
        mat = np.fft.fft(np.eye(N))
        
    return mat

In [ ]:
def naive_dft(signal):
    dft_matrix = construct_dft_matrix(len(signal))
    
    start = time.time()
    np.dot(dft_matrix, signal)
    end = time.time()
    
    return end - start

In [ ]:
# Depending on how much RAM you have, you may need to decrease the maximum 
# exponent here from 16 to something lower.
powers_of_two = [2 ** n for n in range(8, 16)]
time_data = []

for N in powers_of_two:
    signal = random_signal_generator(N)
    time_data.append(naive_dft(signal))

In [ ]:
plt.plot(powers_of_two, time_data)

## Naive FFT with recursion

$$    
\tilde{X}[k] = \sum_{m=0}^{N/2-1} x[2m] e^{-j\frac{2\pi}{N/2} k m} + e^{-jk
          \frac{2\pi}{N}} \sum_{m=0}^{N/2-1} x[2m+1] e^{-j\frac{2\pi}{N/2} k m}
$$

The method below was adapted from a nice [tutorial about the FFT algorithm](https://nbviewer.org/url/jakevdp.github.io/downloads/notebooks/UnderstandingTheFFT.ipynb).

In [ ]:
def naive_fft(signal):
    N = len(signal)

    if N == 1:
        return signal
    
    even_part = signal[::2]
    odd_part = signal[1::2]
    
    even_fft = naive_fft(even_part)
    odd_fft = naive_fft(odd_part)
    
    extra_factors = np.array([np.exp(-1j * k * 2 * np.pi / N) for k in range(N)])
    
    result = np.zeros(N, dtype=np.complex128)
    result[:N//2] = even_fft + extra_factors[:N//2] * odd_fft
    result[N//2:] = even_fft - extra_factors[N//2:] * odd_fft

    return result 

In [ ]:
better_time_data = []

for N in powers_of_two:
    signal = random_signal_generator(N)
    start = time.time()
    naive_fft(signal)
    end = time.time()
    better_time_data.append(end - start)

In [ ]:
plt.plot(powers_of_two, time_data)
plt.plot(powers_of_two, better_time_data)

## NumPy FFT 

In [ ]:
best_time_data = []

for N in powers_of_two:
    signal = random_signal_generator(N)
    start = time.time()
    np.fft.fft(signal)
    end = time.time()
    best_time_data.append(end - start)

In [ ]:
plt.plot(powers_of_two, time_data)
plt.plot(powers_of_two, better_time_data)
plt.plot(powers_of_two, best_time_data)